# Import

In [1]:
from scipy.optimize import fsolve
import numpy as np

# Parameters

In [2]:
def f(I) :
    return X*(1.+I)*(3.+3.*I+I**2)-Y

In [3]:
def deriv_f(I) :
    return X*(3.+3.*I+I**2) + X*(1.+I)*(3.+2.*I)

In [4]:
X = 1000
Y = 3241

# 1-D

## Methode de Newton

In [5]:
I=0.
residu=1
niter=0
DV=[I,I,I,I] #Dernières valeurs
eps = 10e-4

while abs(residu)>eps:
    niter=niter+1
    I=I-f(I)/deriv_f(I)
    residu=f(I)
    DV=[DV[1],DV[2],DV[3],I]
        
print("Taux final avec Newton I=",I,"niter=",niter)
print("Dernières valeurs :",DV)
print("residu :",residu)

Taux final avec Newton I= 0.03913561256092804 niter= 3
Dernières valeurs : [0.0, 0.04016666666666667, 0.03913630481075633, 0.03913561256092804]
residu : 1.9740582501981407e-09


## Methode de la parallele

In [6]:
I=0
residu=1
D=deriv_f(I)
niter=0
DV=[I,I,I,I] #Dernières valeurs

while abs(residu)>1e-15:
    niter=niter+1
    I=I-f(I)/D
    residu=f(I)
    DV=[DV[1],DV[2],DV[3],I]
print("Taux final avec methode de la parallele I=",I,"niter=",niter)
print("Dernières valeurs :",DV)
print("residu :",residu)

Taux final avec methode de la parallele I= 0.03913561256061559 niter= 13
Dernières valeurs : [0.03913561256061226, 0.039135612560615894, 0.03913561256061551, 0.03913561256061559]
residu : 0.0


## Methode fsolve

In [7]:
Isolve=fsolve(f, 0)
print("courant final avec fsolve I=",Isolve)

courant final avec fsolve I= [0.03913561]


In [8]:
def fct1_pt_fixe(I):
    return 1./6.*(Y/X - 3. - 4.*I**2-I**3)

Iold=0
Inew=fct1_pt_fixe(Iold)
residu=1
niter=0
DV=[Iold,Iold,Iold,Inew] #Dernières valeurs

while abs(residu)>1E-10:
    niter=niter+1
    Iold=Inew
    Inew=fct1_pt_fixe(Iold)
    residu=abs((Inew-Iold)/Inew)
    DV=[DV[1],DV[2],DV[3],Inew]
    
print("courant final avec point fixe fonction 1,   I=",Inew)
print("Dernières valeurs :",DV,"niter=",niter)   

courant final avec point fixe fonction 1,   I= 0.03913561256068012
Dernières valeurs : [0.039135612126162206, 0.03913561258361847, 0.039135612559397716, 0.03913561256068012] niter= 8


# Methode point fixe

In [32]:
def g(t):
    return 1./6.*(Y/X - 3.- 4.*t*t-t*t*t)

In [44]:
Iold=3
Inew=g(Iold)
residu=1
niter=0
DV=[Iold,Iold,Iold,Inew] #Dernières valeurs

while abs(residu)>1E-10:
    niter=niter+1
    Iold=Inew
    Inew=g(Iold)
    residu=abs((Inew-Iold)/Inew)
    DV=[DV[1],DV[2],DV[3],Inew]
    
print("courant final avec point fixe fonction 1,   I=",Inew)
print("Dernières valeurs :",DV,"niter=",niter)   

courant final avec point fixe fonction 1,   I= -inf
Dernières valeurs : [3735007813268571.0, -8.684069561041289e+45, 1.0914871580195764e+137, -inf] niter= 6


# 2-D

In [9]:
def Systeme(I):
    Res=np.zeros(2)
    Res[0]=X*(1.+I[0])*(3.+3.*I[0]+I[0]**2)+X*(1.+I[1])*(1.+2.*I[1])-2.*Y
    Res[1]=2.*I[0]-I[1]
    return Res

In [10]:
def Jacobienne(I):
    J=np.zeros([2,2])
    J11=X*(3.+3.*I[0]+I[0]**2) + X*(1.+I[0])*(3.+2.*I[0])
    J12=X*(1.+3.*I[1]) + 2.*X*(1.+I[1])
    J21=2.
    J22=-1.
    J[0,0],J[0,1],J[1,0],J[1,1]=J11,J12,J21,J22
    return J

## Systeme Methode Newton

In [11]:
I=np.zeros(2)
residu=1
niter=1
while residu >1E-10 and niter<1000:
    niter=niter+1
    I=I-np.dot(np.linalg.inv(Jacobienne(I)),Systeme(I))
    residu=np.linalg.norm(Systeme(I))
    
print("solution systeme Newton=",I,"niter=",niter)
print("residu",residu)

solution systeme Newton= [0.17556081 0.35112162] niter= 12
residu 1.0913936421275139e-11


## Systeme Methode Parallele

In [12]:
I=np.zeros(2)
I[0]=1
I[1]=1
J=Jacobienne(I)
invJ=np.linalg.inv(J)
residu=1
niter=1

while residu>1E-10 and niter<1000:
    niter=niter+1
    I=I-np.dot(invJ,Systeme(I))
    residu=np.linalg.norm(Systeme(I))
        
print("solution systeme parallele=",I,"niter=",niter)
print("residu",residu)

solution systeme parallele= [0.17556081 0.35112162] niter= 48
residu 5.184119800105691e-11


## Methode avec fsolve

In [13]:
CI=np.zeros(2)
I=fsolve(Systeme,CI)
residu=np.linalg.norm(Systeme(I))
print("fsolve: I1= %f I2 = %f avec residu %.10f" % (I[0],I[1],residu))

fsolve: I1= 0.175561 I2 = 0.351122 avec residu 0.0000000000
